### Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

#### Methodology

To solve the problem, a Random Forest Classifier was build and the more relevant features of the model were analized using the function 'featureImportances'. The features that best explain the accuracy of the model are the ones expected to be causing the problem that is predicted.

#### Initialize session & imports

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### Read and analize data

In [0]:
data = spark.read.csv('/FileStore/tables/dog_food.csv', inferSchema=True, header=True)

In [0]:
data.show(10)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
+---+---+----+---+-------+
only showing top 10 rows



In [0]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



Column C is composed of float numbers, to investigate if it's important or just a format issue, I am going to check its unique values.

In [0]:
c_unique_values = data.select("C").distinct().show()

+----+
|   C|
+----+
| 8.0|
| 7.0|
|11.0|
|14.0|
|10.0|
|13.0|
| 6.0|
| 5.0|
| 9.0|
|12.0|
+----+



It is just a format issue

#### Feature engineering

Although not strictly necessary, I'm going to convert column 'C' and label 'Spoiled' to integers for clarity.

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [0]:
# The name of the 'Spoiled' column was changed to 'label'
data_cast = data.select(col('A'), col('B'), col("C").cast('int').alias("C"), col('D'), col('Spoiled').cast('int').alias('label'))

In [0]:
data_cast.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: integer (nullable = true)
 |-- D: integer (nullable = true)
 |-- label: integer (nullable = true)



In [0]:
data_cast.show(5)

+---+---+---+---+-----+
|  A|  B|  C|  D|label|
+---+---+---+---+-----+
|  4|  2| 12|  3|    1|
|  5|  6| 12|  7|    1|
|  6|  2| 13|  6|    1|
|  4|  2| 12|  1|    1|
|  4|  2| 12|  3|    1|
+---+---+---+---+-----+
only showing top 5 rows



#### Assembling a 'features' column

Next, I'm going to assemble data from columns 'A', 'B', 'C' and 'D' in a column called 'features'.

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')
data_assembled = assembler.transform(data_cast)

In [0]:
data_assembled.show(5)

+---+---+---+---+-----+------------------+
|  A|  B|  C|  D|label|          features|
+---+---+---+---+-----+------------------+
|  4|  2| 12|  3|    1|[4.0,2.0,12.0,3.0]|
|  5|  6| 12|  7|    1|[5.0,6.0,12.0,7.0]|
|  6|  2| 13|  6|    1|[6.0,2.0,13.0,6.0]|
|  4|  2| 12|  1|    1|[4.0,2.0,12.0,1.0]|
|  4|  2| 12|  3|    1|[4.0,2.0,12.0,3.0]|
+---+---+---+---+-----+------------------+
only showing top 5 rows



For clarity, a create a dataframe called 'final_data' that contains only the relevant columns for the model: 'features' and 'label'

In [0]:
final_data = data_assembled.select('features', 'label')

In [0]:
final_data.show(5)

+------------------+-----+
|          features|label|
+------------------+-----+
|[4.0,2.0,12.0,3.0]|    1|
|[5.0,6.0,12.0,7.0]|    1|
|[6.0,2.0,13.0,6.0]|    1|
|[4.0,2.0,12.0,1.0]|    1|
|[4.0,2.0,12.0,3.0]|    1|
+------------------+-----+
only showing top 5 rows



#### Building a random forest classifier model

First I'm going to split the data in training and test datasets. A train/test split is not necessary for this methodology, but I find it interesting to know the predictive power of the model.


In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

Building of the model

In [0]:
rfc = RandomForestClassifier(labelCol='label', featuresCol='features')

In [0]:
rfc_model = rfc.fit(train_data)

Obtaining the predicitons

In [0]:
rfc_preds = rfc_model.transform(test_data)

Evaluating the model

The Area Under the Curve (AUC) and the average accuracy of the model were measured

In [0]:
binary_eval = BinaryClassificationEvaluator(labelCol='label')
multi_eval = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')

In [0]:
auc = binary_eval.evaluate(rfc_preds)
acc = multi_eval.evaluate(rfc_preds)

In [0]:
print('AUC: ', auc)
print('Accuracy: ', acc)

AUC:  0.9923371647509578
Accuracy:  0.9565217391304348


The model predicts the spoiling of the food extremely well

#### Analysis of the importance of every feature in the models

In [0]:
rfc_model.featureImportances

Out[46]: SparseVector(4, {0: 0.0366, 1: 0.0224, 2: 0.923, 3: 0.018})

Component C (index 2) is clearly the one that explains the accuracy of the model and therefore it is the most probable candidate to be the agent causing the spoiling.